<a href="https://colab.research.google.com/github/i2mmmmm/Study/blob/main/image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
import numpy as np

from google.colab import drive
import os

# Google 드라이브 마운트
drive.mount('/content/drive')

# 이미지가 저장된 경로 설정
image_path = '/content/drive/My Drive/sculpture/'

# 경로에 있는 이미지 파일 목록 가져오기
image_files = os.listdir(image_path)

# 이미지 파일 경로 출력
for image_file in image_files:
    print(os.path.join(image_path, image_file))



# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 100
hidden_size = 64
image_size = 64
num_epochs = 100
batch_size = 64
sample_dir = 'samples'

# Create a directory if not exists
import os
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Define the generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_size, hidden_size * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(hidden_size * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 8, hidden_size * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 4, hidden_size * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 2, hidden_size, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

# Load the dataset
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

dataset = ImageFolder(root='your_dataset_directory', transform=transform)
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Initialize the generator
generator = Generator().to(device)

# Loss function and optimizer
criterion = nn.BCELoss()
optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.to(device)

        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(images.size(0), 1).to(device)
        fake_labels = torch.zeros(images.size(0), 1).to(device)

        # ================================================================== #
        #                      Train the generator                           #
        # ================================================================== #

        # Generate fake images
        z = torch.randn(images.size(0), latent_size, 1, 1).to(device)
        fake_images = generator(z)

        # Compute the generator loss
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)

        # Backprop and optimize
        optimizer_g.zero_grad()
        g_loss.backward()
        optimizer_g.step()

        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Generator Loss: {:.4f}'
                  .format(epoch, num_epochs, i+1, total_step, g_loss.item()))

    # Save sampled images
    if (epoch+1) % 10 == 0:
        save_image(fake_images, os.path.join(sample_dir, 'fake_images-{}.PNG'.format(epoch+1)))

# Save the model checkpoints
torch.save(generator.state_dict(), '/content/sculpture/generator.ckpt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/sculpture/'

In [19]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
import os
from PIL import Image


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 100
hidden_size = 64
image_size = 64
num_epochs = 100
batch_size = 64
sample_dir = '/content/drive/My Drive/generated_images'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)


# 이미지가 저장된 경로 설정
image_path = '/content/sculpture/'

# Custom dataset
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return 20  # Assuming you have 20 images

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, f'{idx + 1}.PNG')
        image = Image.open(img_name).convert('RGB')  # 이미지를 RGB로 변환
        if self.transform:
            image = self.transform(image)
        return image

# Load the dataset
dataset = CustomDataset(root_dir=image_path, transform=transform)
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Define the generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_size, hidden_size * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(hidden_size * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 8, hidden_size * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 4, hidden_size * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 2, hidden_size, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

# Initialize the generator
generator = Generator().to(device)

# Loss function and optimizer
criterion = nn.BCELoss()
optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, images in enumerate(data_loader):
        images = images.to(device)

        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(images.size(0), 1).to(device)
        fake_labels = torch.zeros(images.size(0), 1).to(device)

        # ================================================================== #
        #                      Train the generator                           #
        # ================================================================== #

        # Generate fake images
        z = torch.randn(images.size(0), latent_size, 1, 1).to(device)
        fake_images = generator(z)

        # Compute the generator loss
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)

        # Backprop and optimize
        optimizer_g.zero_grad()
        g_loss.backward()
        optimizer_g.step()

        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Generator Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, g_loss.item()))

    # Save sampled images
    if (epoch+1) % 10 == 0:
        save_image(fake_images, os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints
torch.save(generator.state_dict(), '/content/sculpture/generator.ckpt')

NameError: name 'discriminator' is not defined

In [31]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
import os
from PIL import Image
from google.colab import drive

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 100
hidden_size = 64
image_size = 64
num_epochs = 100
batch_size = 64
sample_dir = '/content/sculpture'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)


# 이미지가 저장된 경로 설정
image_path = '/content/sculpture/'

# Custom dataset
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return 20  # Assuming you have 20 images

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, f'{idx + 1}.png')
        image = Image.open(img_name).convert('RGB')  # 이미지를 RGB로 변환
        if self.transform:
            image = self.transform(image)
        return image

# Load the dataset
dataset = CustomDataset(root_dir=image_path, transform=transform)
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Define the generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_size, hidden_size * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(hidden_size * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 8, hidden_size * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 4, hidden_size * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 2, hidden_size, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

# Initialize the generator
generator = Generator().to(device)

# Loss function and optimizer
criterion = nn.BCELoss()

# Training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, real_images in enumerate(data_loader):
        real_images = real_images.to(device)

        # Create the labels for real and fake images
        real_labels = torch.ones(real_images.size(0), 1).to(device)
        fake_labels = torch.zeros(real_images.size(0), 1).to(device)

        # Train Generator
        optimizer_g.zero_grad()

        # Generate fake images
        z = torch.randn(real_images.size(0), latent_size, 1, 1).to(device)
        fake_images = generator(z)

        # Compute the generator loss
        g_loss = criterion(discriminator(fake_images), real_labels)

        # Backpropagation
        g_loss.backward()
        optimizer_g.step()

        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Generator Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, g_loss.item()))

    # Save sampled images
    if (epoch+1) % 10 == 0:
        save_image(fake_images, os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-31-3afe1febdeb5>", line 47, in __getitem__
    image = Image.open(img_name).convert('RGB')  # 이미지를 RGB로 변환
  File "/usr/local/lib/python3.10/dist-packages/PIL/Image.py", line 3227, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '/content/sculpture/1.png'


In [28]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
import os
from PIL import Image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 100
hidden_size = 64
image_size = 64
num_epochs = 100
batch_size = 4  # 작은 배치 크기 사용
sample_dir = '/content/drive/My Drive/generated_images'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# 이미지가 저장된 경로 설정
image_path = '/content/sculpture/'

# Custom dataset
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = os.listdir(root_dir)  # 폴더 내의 모든 이미지 파일을 리스트로 가져옴

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_name).convert('RGB')  # 이미지를 RGB로 변환
        if self.transform:
            image = self.transform(image)
        return image

# Load the dataset
dataset = CustomDataset(root_dir=image_path, transform=transform)
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Define the generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_size, hidden_size * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(hidden_size * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 8, hidden_size * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 4, hidden_size * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size * 2, hidden_size, 4, 2, 1, bias=False),
            nn.BatchNorm2d(hidden_size),
            nn.ReLU(True),
            nn.ConvTranspose2d(hidden_size, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

# Initialize the generator
generator = Generator().to(device)

# Loss function and optimizer
criterion = nn.BCELoss()
optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, images in enumerate(data_loader):
        images = images.to(device)

        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(images.size(0), 1).to(device)
        fake_labels = torch.zeros(images.size(0), 1).to(device)

        # Generate fake images
        z = torch.randn(images.size(0), latent_size, 1, 1).to(device)
        fake_images = generator(z)

        # Compute the generator loss
        g_loss = criterion(fake_images, real_labels.unsqueeze(1).unsqueeze(2).unsqueeze(3).expand(-1, 3, -1, -1))

        # Backprop and optimize
        optimizer_g.zero_grad()
        g_loss.backward()
        optimizer_g.step()

        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Generator Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, g_loss.item()))

    # Save sampled images
    if (epoch+1) % 10 == 0:
        save_image(fake_images, os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints
torch.save(generator.state_dict(), '/content/sculpture/generator.ckpt')

IsADirectoryError: Caught IsADirectoryError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-28-ff04f6ad9271>", line 46, in __getitem__
    image = Image.open(img_name).convert('RGB')  # 이미지를 RGB로 변환
  File "/usr/local/lib/python3.10/dist-packages/PIL/Image.py", line 3227, in open
    fp = builtins.open(filename, "rb")
IsADirectoryError: [Errno 21] Is a directory: '/content/sculpture/.ipynb_checkpoints'


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
import os
from PIL import Image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 100
hidden_size = 64
image_size = 64
num_epochs = 100
batch_size = 4  # 작은 배치 크기 사용
sample_dir = '/content/drive/My Drive/generated_images'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# 이미지가 저장된 경로 설정
image_path = '/content/sculpture/'

# Custom dataset
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = [f for f in os.listdir(root_dir) if f.endswith('.png')]  # .png 확장자를 가진 파일만 가져옴

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_name).convert('RGB')  # 이미지를 RGB로 변환
        if self.transform:
            image = self.transform(image)
        return image

# Load the dataset
dataset = CustomDataset(root_dir=image_path, transform=transform)
data_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)  # num_workers를 0으로 설정하여 멀티프로세싱을 비활성화함

# 나머지 코드는 동일합니다.